# Data Cleaning

Real world data is usually messy and can contain errors. These can be in the form of unnecessary (sometimes empty) columns, values not formatted correctly or recorded erroneously.

In this notebook we present a few techniques on how to tackle these issues. We will use the *student_debt* data as previously.

In [64]:
import pandas as pd

df = pd.read_csv("student_debt.csv")
df

,Unnamed: 0,Period,Characteristic,No.people,Sum,Average,Median
0,2,2011,Total,754.2,9.5,12.6,7.4
1,3,2011,Man,391.0,5.1,13.1,7.6
2,4,2011,Woman,363.1,4.4,12.0,7.2
3,5,2011,up to 20 years old,41.1,0.1,2.5,1.4
4,6,2011,between 20 and 25 years old,284.8,2.3,8.1,4.8
...,...,...,...,...,...,...,...
67,69,2019*,up to 20 years old,104.5,0.4,4.1,2.5
68,70,2019*,between 20 and 25 years old,479.0,5.2,10.9,7.2
69,71,2019*,between 25 and 45 years old,822.2,13.6,16.5,10.6
70,72,2019*,between 45 and 65 years old,7.7,0.1,12.4,6.2


Notice the column `Unnamed: 0` does not contain any useful information, it is a mismatched version of the index only. This usually happens when the data is misformatted. 

We can easily get rid of a column in Pandas with the `drop` function. The drop function accepts a list of column names in the columns attribute. 
Furthermore, you can pass `inplace = True` to perform the operation on the dataframe. If you do not do this, the operation returns a new dataframe without the specified columns.

In [65]:
df_dropped_column = df.drop(columns=["Unnamed: 0"]) #returns a new dataframe 
df.drop(columns=["Unnamed: 0"], inplace=True)

Now that we have only the useful columns, we can look at the values in our data set. 
For this a useful starting point is looking at whether all data types are as expected. 

In [66]:
df.dtypes

Period             object
Characteristic     object
No.people         float64
Sum               float64
Average           float64
Median            float64
dtype: object

Notice that `Period` has a data type object, whereas it should be an integer or float. 
If you look at some of the year values, they have an asterisk * next to them. This is because those figures are preliminary only, but we still need those years in raw number format, such that we can perform analysis on it.

We can use the `.apply()` function. This applies a custom function on every row of the dataframe its called upon.
For simpler functions we can write a lambda function. Don't let the fancy name fool you, they are actually pretty easy.

In python we can call the `replace()` method to get rid of the * after the years. Then we can convert the years to integers with `int()`.

In [67]:
# we specifiy a lambda function which applies a certain operation on every row of our dataframe
df.apply(lambda row:int(row['Period'].replace('*', '')), axis=1) 
#axis = 1 needed to apply the operations on every row, axis = 0 would apply the operations on every column


0     2011
1     2011
2     2011
3     2011
4     2011
      ... 
67    2019
68    2019
69    2019
70    2019
71    2019
Length: 72, dtype: int64

Notice that it returns a new Pandas series, and does not modify the original dataframe. We can create or replace a column with pandas by simply specifying the column name and the values of the new column.

In [68]:
df["Period"] = df.apply(lambda row:int(row['Period'].replace('*', '')), axis=1) 
df.dtypes

Period              int64
Characteristic     object
No.people         float64
Sum               float64
Average           float64
Median            float64
dtype: object

Notice that the data type of Period now changed to int64, which was our desired goal.

# Missing values and outliers on Salary data set

In some data sets you have missing or otherwise erroneous values. In the data dashboard there are only data sets without these issues, but in general, it is a very common and important step in data cleaning.

For this we will use a different data set downloaded from our data dashboard.

In [144]:
df_salaries = pd.read_csv("DenBosch-avg-salary.csv")
df_salaries.iloc[40:50]

,AreaCode,Neighbourhood code,Total avg. salary (thousand EUR per year),<25yrs old,25-44yrs old,45-64yrs old,65+ yrs old
40,06,Totaal,"35,4","11,1","42,3","46,3","25,6"
41,06,01,"33,7","10,9","44,1","43,0","24,8"
42,06,02,"32,0","12,9","42,3","40,4","24,9"
43,06,03,"34,8","10,8","41,3","46,2","24,9"
44,06,04,"38,5",.,.,"50,1",.
45,06,05,.,.,.,.,.
46,06,06,"36,0","9,3","40,8","46,3","26,6"
47,06,07,"38,7","11,6","42,6","50,2","28,4"
48,06,08,.,.,.,.,.
49,06,09,.,.,.,.,.


In [134]:
df_salaries.dtypes

AreaCode                                     object
Neighbourhood code                           object
Total avg. salary (thousand EUR per year)    object
<25yrs old                                   object
25-44yrs old                                 object
45-64yrs old                                 object
65+ yrs old                                  object
dtype: object

As you can see, part of this data has missing values (in the form of `.`) in several places. Furthermore also notice that all the numerical values are listed as objects, because Python uses a `.` to indicate decimals, and in this data set we have `,` separating them. 

First we have to do two things: replace the `,` to `.` in order to convert the numbers to `float`, and set the missing values to be actually missing, this latter can be done by filtering cells containing a `.` and setting their values to `pd.NA`.

For this exercise, we select only the columns we want to clean. In our case, we only want columns with numbers, so we can drop *AreaCode, Neighbourhood code* columns for imputation. 

Later on, we need to recombine (concatenate) the imputed columns with the other two we decided to drop.

In [139]:
import numpy as np 

columns_not_impute = ['AreaCode', 'Neighbourhood code'] # columns we don't want to preprocess further
df_salaries_to_impute = df_salaries.drop(columns=columns_not_impute)

#iterate over all columns and replace , with . using a lambda function
for column in df_salaries_to_impute.columns:
    df_salaries_to_impute[column] = df_salaries_to_impute.apply(lambda row:row[column].replace(',', '.'), axis=1)

df_salaries_to_impute[df_salaries_to_impute.values == '.'] = np.nan #set the missing values to NaN

df_salaries_to_impute = df_salaries_to_impute.astype(float)


Now with the missing values we can do something called *missing value imputation*, which replaces missing values with some plausible value.
For this we will use the Python library scikit-learn, that has multiple methods to impute missing values.

In [122]:
df_salaries_to_impute.dtypes

Total avg. salary (thousand EUR per year)    object
<25yrs old                                   object
25-44yrs old                                 object
45-64yrs old                                 object
65+ yrs old                                  object
dtype: object

In [121]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer() #initialize scikit learn imputer
imputer.fit_transform(df_salaries_to_impute)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '35,4'